### 3. Programación lineal, método simplex

In [2]:
import numpy as np
from fractions import Fraction

print("\n                 Metodo Simplex\n\n")

# Coeficiente de las restricciones (matriz)
A = np.array([[3, 1], [1, 4]])

# Cantidad de recursos (arreglo b)
b = np.array([8, 8])

# Coeficientes de la funcion obketivo (arreglo c)
c = np.array([50, 40])

# Variables básicas (matriz)
B = np.array([2, 3])  # Ajusta los indices de las variables básicas

# Coeficientes de las variables básicas en la función objetivo (arreglo cb)
cb = np.array([c[i] if i in B else 0 for i in range(len(c))])

# Valores inciales de las variables básicas (arreglo xb)
xb = np.transpose([b])

# Construcción de la tabla simplez inicial
table = np.hstack((B.reshape(-1, 1), cb.reshape(-1, 1)))
table = np.hstack((table, xb))
table = np.hstack((table, A))
table = np.array(table, dtype='float')

# Flag para problemas de minimización
MIN = 0

print("Tabla en iteración = 0")
print("B \tCB \tXB \tx1 \tx2")
for row in table:
    for el in row:
        print(Fraction(str(el)).limit_denominator(100), end='\t')
    print()
print()
print("Simplex corriendo....")

# Variables de iteración iniciales
reached = 0
itr = 1
unbounded = 0
alternate = 0

while reached == 0:

    print("Iteración: ", end=' ')
    print(itr)
    print("B \tCB \tXB \tx1 \tx2")
    for row in table:
        for el in row:
            print(Fraction(str(el)).limit_denominator(100), end='\t')
        print()

    # Calculamos la utilidad relativa -> cj - zj para no basicas
    rel_prof = c - np.sum(table[:, 1].reshape(-1, 1) * table[:, 3:], axis=1)

    print("Utilidad relativa: ", end=" ")
    for profit in rel_prof:
        print(Fraction(str(profit)).limit_denominator(100), end=", ")
    print()

    b_var = table[:, 0]
    # Buscamos soluciones alternativas
    for i in range(len(c)):
        if i not in b_var:
            if rel_prof[i] == 0:
                alternate = 1
                print("Escenario alterno encontrado")
                break

    print()
    flag = 0
    for profit in rel_prof:
        if profit > 0:
            flag = 1
            break

    # Si todas las utilidades relativas son <= 0, óptimo alcanzado
    if flag == 0:
        print("Todas las utilidades son <= 0, óptimo alcanzado")
        reached = 1
        break

    # ingresamos la variable
    k = rel_prof.argmax()

    # prueba de razón mínima (sólo valores positivos)
    min_ratio = 99999
    r = -1
    for i in range(len(table)):
        if table[:, 2][i] > 0 and table[:, 3 + k][i] > 0:
            val = table[:, 2][i] / table[:, 3 + k][i]
            if val < min_ratio:
                min_ratio = val
                r = i  # variable que se conserva

    # Si no se ejecuta la prueba de razón mínima
    if r == -1:
        unbounded = 1
        print("Caso sin  ilimitado")
        break

    print("Indice elemento pivote:", end=' ')
    print(np.array([r, 3 + k]))

    pivot = table[r][3 + k]
    print("Elemento pivote: ", end=" ")
    print(Fraction(pivot).limit_denominator(100))

    # Ejecutamos las operaciones por columna
    # Dividimos la columna pivote con los elementos pivote
    table[r, 2:] = table[r, 2:] / pivot

    # Hacer las operaciones por columna en otra columna
    for i in range(len(table)):
        if i != r:
            table[i, 2:] = table[i, 2:] - table[i][3 + k] * table[r, 2:]

    # Asignamos la nueva variable básica
    table[r][0] = k
    table[r][1] = c[k]

    print()
    itr += 1

print()

print("---------------------------------------------------------------")
if unbounded == 1:
    print("ILIMITADO LPP")
    exit()
if alternate == 1:
    print("Solución alternativa")

print("Tabla óptima:")
print("B \tCB \tXB \tx1 \tx2")
for row in table:
    for el in row:
        print(Fraction(str(el)).limit_denominator(100), end='\t')
    print()
print()
print("Valor z en su punto óptimo: ", end=" ")

basis = []
sum_value = 0
for i in range(len(table)):
    sum_value += c[int(table[i][0])] * table[i][2]
    temp = "x" + str(int(table[i][0]) + 1)
    basis.append(temp)

# If MIN problem make z negative
if MIN == 1:
    print(-Fraction(str(sum_value)).limit_denominator(100))
else:
    print(Fraction(str(sum_value)).limit_denominator(100))
print("Base Final: ", end=" ")
print(basis)

print("Simplex Finalizado")
print()


                 Metodo Simplex


Tabla en iteración = 0
B 	CB 	XB 	x1 	x2
2	0	8	3	1	
3	0	8	1	4	

Simplex corriendo....
Iteración:  1
B 	CB 	XB 	x1 	x2
2	0	8	3	1	
3	0	8	1	4	
Utilidad relativa:  50, 40, 

Indice elemento pivote: [0 3]
Elemento pivote:  3

Iteración:  2
B 	CB 	XB 	x1 	x2
0	50	8/3	1	1/3	
3	0	16/3	0	11/3	
Utilidad relativa:  -50/3, 40, 

Indice elemento pivote: [1 4]
Elemento pivote:  11/3

Iteración:  3
B 	CB 	XB 	x1 	x2
0	50	24/11	1	0	
1	40	16/11	0	1	
Utilidad relativa:  0, 0, 

Todas las utilidades son <= 0, óptimo alcanzado

---------------------------------------------------------------
Tabla óptima:
B 	CB 	XB 	x1 	x2
0	50	24/11	1	0	
1	40	16/11	0	1	

Valor z en su punto óptimo:  1840/11
Base Final:  ['x1', 'x2']
Simplex Finalizado

